# Understanding Hired Rides in NYC

_[Project prompt](https://docs.google.com/document/d/1VERPjEZcC1XSs4-02aM-DbkNr_yaJVbFjLJxaYQswqA/edit#)_

_This scaffolding notebook may be used to help setup your final project. It's **totally optional** whether you make use of this or not._

_If you do use this notebook, everything provided is optional as well - you may remove or add prose and code as you wish._

_Anything in italics (prose) or comments (in code) is meant to provide you with guidance. **Remove the italic lines and provided comments** before submitting the project, if you choose to use this scaffolding. We don't need the guidance when grading._

_**All code below should be consider "pseudo-code" - not functional by itself, and only a suggestion at the approach.**_

## Requirements

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project._

* Code clarity: make sure the code conforms to:
    * [ ] [PEP 8](https://peps.python.org/pep-0008/) - You might find [this resource](https://realpython.com/python-pep8/) helpful as well as [this](https://github.com/dnanhkhoa/nb_black) or [this](https://jupyterlab-code-formatter.readthedocs.io/en/latest/) tool
    * [ ] [PEP 257](https://peps.python.org/pep-0257/)
    * [ ] Break each task down into logical functions
* The following files are submitted for the project (see the project's GDoc for more details):
    * [ ] `README.md`
    * [ ] `requirements.txt`
    * [ ] `.gitignore`
    * [ ] `schema.sql`
    * [ ] 6 query files (using the `.sql` extension), appropriately named for the purpose of the query
    * [x] Jupyter Notebook containing the project (this file!)
* [x] You can edit this cell and add a `x` inside the `[ ]` like this task to denote a completed task

## Project Setup

In [88]:
# import libraries

from math import radians, sin, cos, sqrt, atan2

import os
import requests
import bs4
import re
import geopandas
import matplotlib.pyplot as plt
import pandas as pd
import sqlalchemy as db

<IPython.core.display.Javascript object>

In [104]:
# notebook formatting

# %load_ext nb_black
%reload_ext nb_black

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [3]:
# any constants you might need, for example:

TAXI_URL = "https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page"
# add other constants to refer to any local data, e.g. uber & weather
UBER_CSV = "uber_rides_sample.csv"

NEW_YORK_BOX_COORDS = ((40.560445, -74.242330), (40.908524, -73.717047))

DATABASE_URL = "sqlite:///project.db"
DATABASE_SCHEMA_FILE = "schema.sql"
QUERY_DIRECTORY = "queries"

<IPython.core.display.Javascript object>

## Part 1: Data Preprocessing

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] Define a function that calculates the distance between two coordinates in kilometers that **only uses the `math` module** from the standard library.
* [ ] Taxi data:
    * [ ] Use the `re` module, and the packages `requests`, BeautifulSoup (`bs4`), and (optionally) `pandas` to programmatically download the required CSV files & load into memory.
    * You may need to do this one file at a time - download, clean, sample. You can cache the sampling by saving it as a CSV file (and thereby freeing up memory on your computer) before moving onto the next file. 
* [ ] Weather & Uber data:
    * [ ] Download the data manually in the link provided in the project doc.
* [ ] All data:
    * [ ] Load the data using `pandas`
    * [ ] Clean the data, including:
        * Remove unnecessary columns
        * Remove invalid data points (take a moment to consider what's invalid)
        * Normalize column names
        * (Taxi & Uber data) Remove trips that start and/or end outside the designated [coordinate box](http://bboxfinder.com/#40.560445,-74.242330,40.908524,-73.717047)
    * [ ] (Taxi data) Sample the data so that you have roughly the same amount of data points over the given date range for both Taxi data and Uber data.
* [ ] Weather data:
    * [ ] Split into two `pandas` DataFrames: one for required hourly data, and one for the required daily daya.
    * [ ] You may find that the weather data you need later on does not exist at the frequency needed (daily vs hourly). You may calculate/generate samples from one to populate the other. Just document what you’re doing so we can follow along. 

    """Summary line.

    Extended description of function.

    Args:
        arg1 (int): Description of arg1
        arg2 (str): Description of arg2

    Returns:
        bool: Description of return value

    """

### Calculating distance
_**TODO:** Write some prose that tells the reader what you're about to do here._

In [4]:
def calculate_distance(from_coord, to_coord):
    """Caculate the distance between two coordinates

    Args:
        from_coord (tuple): A pair values (float) of latitude and longitude
        to_coord (tuple): A pair values (float) of latitude and longitude

    Returns:
        float: the distance, round to 3 decimals

    """

    # convert degrees to radians
    lat1, lon1 = radians(from_coord[0]), radians(from_coord[1])
    lat2, lon2 = radians(to_coord[0]), radians(to_coord[1])

    # apply Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    r = 6373.0  # radius of earth (km)

    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = round(r * c, 3)

    return distance

<IPython.core.display.Javascript object>

In [67]:
def calculate_distance(row):
    """Caculate the distance between two coordinates

    Args:
        from_coord (tuple): A pair values (float) of latitude and longitude
        to_coord (tuple): A pair values (float) of latitude and longitude

    Returns:
        float: the distance, round to 3 decimals

    """

    # convert degrees to radians
    lat1, lon1 = radians(row["pickup_latitude"]), radians(row["pickup_longitude"])
    lat2, lon2 = radians(row["dropoff_latitude"]), radians(row["dropoff_longitude"])

    # apply Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    r = 6373.0  # radius of earth (km)

    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = round(r * c, 3)

    return distance

<IPython.core.display.Javascript object>

In [71]:
# calculate_distance(foo.iloc[0])
# foo.iloc[0]

3.932

<IPython.core.display.Javascript object>

In [76]:
def add_distance_column(dataframe):
    dataframe["distance"] = dataframe.apply(calculate_distance, axis=1)

    return dataframe

<IPython.core.display.Javascript object>

In [79]:
# foo1 = add_distance_column(foo)
# foo1

,pickup_datetime,dropoff_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,tip_amount,distance
0,2010-01-23 02:03:00,2010-01-23 02:12:00,-73.949508,40.776643,-73.974307,40.746695,1.0,3.932
1,2010-01-14 07:53:27,2010-01-14 08:02:49,-73.946638,40.780682,-73.952340,40.802278,0.0,2.450
2,2010-01-14 07:24:00,2010-01-14 07:37:00,-73.949312,40.772953,-73.985162,40.732343,0.0,5.434
3,2010-01-15 10:37:09,2010-01-15 10:54:53,-73.964136,40.776502,-74.005351,40.739304,0.0,5.402
4,2010-01-04 21:39:24,2010-01-04 21:40:01,-73.988878,40.817326,-73.988878,40.817326,15.0,0.000
...,...,...,...,...,...,...,...,...
2495,2010-01-15 09:29:00,2010-01-15 09:37:00,-73.995893,40.763018,-73.986055,40.756852,0.0,1.076
2496,2010-01-10 11:33:00,2010-01-10 11:41:00,-73.979368,40.762188,-73.985425,40.749822,0.0,1.467
2497,2010-01-18 23:04:00,2010-01-18 23:16:00,-73.962440,40.758090,-73.996950,40.742130,2.5,3.407
2498,2010-01-10 16:01:56,2010-01-10 16:05:30,-73.953695,40.766554,-73.959245,40.761876,0.0,0.700


<IPython.core.display.Javascript object>

### Processing Taxi Data

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [9]:
def find_taxi_parquet_urls():
    """Retrieve the yellow taxi parquet urls.

    The function retrieves the url from January 2009 through June 2015.

    Args:
        None

    Returns:
        list: Desired urls as list of strings

    """
    # Use requests library to get TAXI_URL html
    response = requests.get(TAXI_URL)
    html = response.content

    # Use BeautifulSoup to get all yellow taxi parquet urls
    soup = bs4.BeautifulSoup(html, "html.parser")
    yellow_a_tags = soup.find_all("a", attrs={"title": "Yellow Taxi Trip Records"})
    yellow_taxi_urls = [a["href"] for a in yellow_a_tags]

    # Use re moduel to filter the urls
    pattern = re.compile(
        r"yellow_tripdata_20(09-(?:0\d|1[0-2])|1[0-4]-(?:0\d|1[0-2])|15-0[1-6]).parquet"
    )

    yellow_taxi_urls_desired = []

    for link in yellow_taxi_urls:
        match = pattern.search(link)
        if match:
            yellow_taxi_urls_desired.append(match.string)

    return yellow_taxi_urls_desired

<IPython.core.display.Javascript object>

In [58]:
test = find_taxi_parquet_urls()
len(test)

78

<IPython.core.display.Javascript object>

In [10]:
# Deprecated
# def download_taxi_data(taxi_parquet_url):
#     for url in taxi_parquet_url:

#         response = requests.get(url, stream=True)
#         file_name = url.split("/")[-1]
#         # file_path = os.getcwd() + '/datasets/yellow_taxi_data/'

#         with open(file_name, "wb") as f:
#             for chunk in response.iter_content(chunk_size=1024):
#                 if chunk:
#                     f.write(chunk)
# Download the taxi parquet files to the current folder

# taxi_parquet_url = find_taxi_parquet_urls()
# download_taxi_data(taxi_parquet_url)

<IPython.core.display.Javascript object>

In [99]:
columns_needed1 = [
    "Trip_Pickup_DateTime",
    "Trip_Dropoff_DateTime",
    "Start_Lon",
    "Start_Lat",
    "End_Lon",
    "End_Lat",
    "Tip_Amt",
]  # 2009-02

columns_needed2 = [
    "tpep_pickup_datetime",
    "tpep_dropoff_datetime",
    "PULocationID",
    "DOLocationID",
    "tip_amount",
]  # 2014-02

columns_needed3 = [
    "pickup_datetime",
    "dropoff_datetime",
    "pickup_longitude",
    "pickup_latitude",
    "dropoff_longitude",
    "dropoff_latitude",
    "tip_amount",
]  # 2010-01

df = pd.read_parquet("yellow_tripdata_2015-01.parquet")


df.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

<IPython.core.display.Javascript object>

In [100]:
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2015-01-01 00:11:33,2015-01-01 00:16:48,1,1.0,1,N,41,166,1,5.7,0.5,0.5,1.40,0.0,0.0,8.40,None,None
1,1,2015-01-01 00:18:24,2015-01-01 00:24:20,1,0.9,1,N,166,238,3,6.0,0.5,0.5,0.00,0.0,0.0,7.30,None,None
2,1,2015-01-01 00:26:19,2015-01-01 00:41:06,1,3.5,1,N,238,162,1,13.2,0.5,0.5,2.90,0.0,0.0,17.40,None,None
3,1,2015-01-01 00:45:26,2015-01-01 00:53:20,1,2.1,1,N,162,263,1,8.2,0.5,0.5,2.37,0.0,0.0,11.87,None,None
4,1,2015-01-01 00:59:21,2015-01-01 01:05:24,1,1.0,1,N,236,141,3,6.0,0.5,0.5,0.00,0.0,0.0,7.30,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12741030,1,2015-01-31 23:21:42,2015-01-31 23:31:00,1,1.6,1,N,90,249,1,8.0,0.5,0.5,2.32,0.0,0.3,11.62,None,None
12741031,1,2015-01-31 23:42:43,2015-01-31 23:49:32,1,0.6,1,N,90,68,1,6.0,0.5,0.5,1.46,0.0,0.3,8.76,None,None
12741032,1,2015-01-31 23:55:16,2015-02-01 00:16:45,1,3.0,1,N,68,148,1,15.0,0.5,0.5,4.07,0.0,0.3,20.37,None,None
12741033,1,2015-01-31 23:20:53,2015-02-01 00:07:35,1,6.9,1,N,189,237,1,32.5,0.5,0.5,6.00,0.0,0.3,39.80,None,None


<IPython.core.display.Javascript object>

### Load taxi zones shp file and convert the polygon to coordinates

In [12]:
def convert_geometry(polygon):
    """Conver polygon to its center point

    Args:
        polygon (Polygon): shapely.geometry.polygon.Polygon type

    Returns:
        tuple: Contain the latitude and longitude values

    """
    lon = polygon.centroid.x
    lat = polygon.centroid.y
    return (lat, lon)


gdf = geopandas.read_file("taxi_zones/taxi_zones.shp").to_crs(4326)
# gdf["center_lon"] = gdf["geometry"].apply(lambda p: p.centroid.x)
# gdf["center_lat"] = gdf["geometry"].apply(lambda p: p.centroid.y)
gdf["location_coordinate"] = gdf["geometry"].apply(convert_geometry)

# Only keep the ID and coordinate columns
gdf = gdf[["LocationID", "location_coordinate"]]

# Remove duplicate LocationID: 56 and 103
gdf = gdf[~gdf["LocationID"].duplicated(keep=False)]

<IPython.core.display.Javascript object>

In [105]:
def clean_taxi_data_with_locationId(df):
    # Keep only the columns that matters
    df = df[
        [
            "tpep_pickup_datetime",
            "tpep_dropoff_datetime",
            "PULocationID",
            "DOLocationID",
            "tip_amount",
        ]
    ]
    # Drop rows where pickup location ID is the same as drop-off
    df = df[df["PULocationID"] != df["DOLocationID"]]

    # TODO: test dropoff Location ID == 56 & 103 (both ID have mutilple geometry entries)
#     df = df.drop(
#         df[
#             (df.PULocationID == 56)
#             | (df.PULocationID == 103)
#             | (df.DOLocationID == 56)
#             | (df.DOLocationID == 103)
#         ].index
#     )

    # Convert LocationID to coordinate
    df["pickup_location"] = df["PULocationID"].map(
        gdf.set_index("LocationID")["location_coordinate"]
    )
    df["dropoff_location"] = df["DOLocationID"].map(
        gdf.set_index("LocationID")["location_coordinate"]
    )
    # Split the coordinate into latitude and longitude
    df[["pickup_latitude", "pickup_longitude"]] = pd.DataFrame(
        df["pickup_location"].tolist(), index=df.index
    )
    df[["dropoff_latitude", "dropoff_longitude"]] = pd.DataFrame(
        df["dropoff_location"].tolist(), index=df.index
    )

    # Drop LocationID columns
    df = df.drop(
        columns=["PULocationID", "DOLocationID", "pickup_location", "dropoff_location"]
    ).reset_index(drop=True)

    # Normalize column names
    df = df.rename(
        columns={
            "tpep_pickup_datetime": "pickup_datetime",
            "tpep_dropoff_datetime": "dropoff_datetime",
        }
    )

    return df

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [96]:
def get_and_clean_month_taxi_data(url):
    response = requests.get(url, stream=True)
    file_name = url.split("/")[-1]

    # Download file if it does not exist
    if not os.path.isfile(file_name):
        with open(file_name, "wb") as f:
            for chunk in response.iter_content(chunk_size=1024):
                if chunk:
                    f.write(chunk)

    df = pd.read_parquet(file_name)

    # There are three dataset types for taxi data
    # 1. Has location ID
    if "PULocationID" in df.columns:
        df = clean_taxi_data_with_locationId(df)

    # 2. Column names with "Start_Lon", "Start_Lat", "End_Lon", "End_Lat"
    elif "Start_Lon" in df.columns:
        # Keep columns we need
        df = df[
            [
                "Trip_Pickup_DateTime",
                "Trip_Dropoff_DateTime",
                "Start_Lon",
                "Start_Lat",
                "End_Lon",
                "End_Lat",
                "Tip_Amt",
            ]
        ]
        # Normalize columns
        df = df.rename(
            columns={
                "Trip_Pickup_DateTime": "pickup_datetime",
                "Trip_Dropoff_DateTime": "dropoff_datetime",
                "Start_Lon": "pickup_longitude",
                "Start_Lat": "pickup_latitude",
                "End_Lon": "dropoff_longitude",
                "End_Lat": "dropoff_latitude",
                "Tip_Amt": "tip_amount",
            }
        )

    # 3. Column names with "pickup_datetime" etc. which is our standard
    elif "pickup_datetime" in df.columns:
        # Keep columns we need
        df = df[
            [
                "pickup_datetime",
                "dropoff_datetime",
                "pickup_longitude",
                "pickup_latitude",
                "dropoff_longitude",
                "dropoff_latitude",
                "tip_amount",
            ]
        ]

    else:
        raise ("Dataset with unknown columns naming standard")

    # Drop out-off-region rows
    df = df.drop(
        df[
            (df.pickup_latitude > 40.908524)
            | (df.pickup_latitude < 40.560445)
            | (df.pickup_longitude > -73.717047)
            | (df.pickup_longitude < -74.242330)
            | (df.dropoff_latitude > 40.908524)
            | (df.dropoff_latitude < 40.560445)
            | (df.dropoff_longitude > -73.717047)
            | (df.dropoff_longitude < -74.242330)
        ].index
    )

    # Convert pickup_datetime and dropoff_datetime to pandas DateTime
    df["pickup_datetime"] = pd.to_datetime(df["pickup_datetime"])
    df["dropoff_datetime"] = pd.to_datetime(df["dropoff_datetime"])

    # Sampling:
    # sampling size = Uber data size / number of months of taxi datasets
    df = df.sample(n=int(195000 / 78))

    return df.reset_index(drop=True)

<IPython.core.display.Javascript object>

In [91]:
os.path.isfile("yellow_tripdata_2013-05.parquet")
# foo = get_and_clean_month_taxi_data(
#     "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2010-01.parquet"
# )

False

<IPython.core.display.Javascript object>

In [75]:
foo.dtypes

pickup_datetime      datetime64[ns]
dropoff_datetime     datetime64[ns]
pickup_longitude            float64
pickup_latitude             float64
dropoff_longitude           float64
dropoff_latitude            float64
tip_amount                  float64
dtype: object

<IPython.core.display.Javascript object>

In [60]:
int(200000 / 78)

2564

<IPython.core.display.Javascript object>

In [86]:
def get_and_clean_taxi_data():
    all_taxi_dataframes = []

    all_csv_urls = find_taxi_parquet_urls()
    for csv_url in all_csv_urls:
        # maybe: first try to see if you've downloaded this exact
        # file already and saved it before trying again
        dataframe = get_and_clean_month_taxi_data(csv_url)
        add_distance_column(dataframe)
        # maybe: if the file hasn't been saved, save it so you can
        # avoid re-downloading it if you re-run the function

        all_taxi_dataframes.append(dataframe)

    # create one gigantic dataframe with data from every month needed
    taxi_data = pd.concat(all_taxi_dataframes)
    return taxi_data

<IPython.core.display.Javascript object>

In [97]:
taxi_data = get_and_clean_taxi_data()

<IPython.core.display.Javascript object>

In [102]:
taxi_data["dropoff_latitude"].isna().sum()

554

<IPython.core.display.Javascript object>

### Processing Uber Data

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [53]:
def load_and_clean_uber_data(csv_file):
    # Load and filter the columns
    df = pd.read_csv(
        csv_file,
        usecols=[
            "pickup_datetime",
            "pickup_longitude",
            "pickup_latitude",
            "dropoff_longitude",
            "dropoff_latitude",
        ],
    )

    # Drop out-off-region rows
    df = df.drop(
        df[
            (df.pickup_latitude > 40.908524)
            | (df.pickup_latitude < 40.560445)
            | (df.pickup_longitude > -73.717047)
            | (df.pickup_longitude < -74.242330)
            | (df.dropoff_latitude > 40.908524)
            | (df.dropoff_latitude < 40.560445)
            | (df.dropoff_longitude > -73.717047)
            | (df.dropoff_longitude < -74.242330)
        ].index
    )

    # Convert pickup_datetime column to pandas DateTime
    df["pickup_datetime"] = pd.to_datetime(df["pickup_datetime"])
    # Convert UTC to EST
    df["pickup_datetime"] = df["pickup_datetime"].dt.tz_convert("US/Eastern")
    # Drop timezone suffix
    df["pickup_datetime"] = df["pickup_datetime"].dt.tz_localize(None)

    return df

<IPython.core.display.Javascript object>

In [54]:
# Test
foo = load_and_clean_uber_data(UBER_CSV)

<IPython.core.display.Javascript object>

In [55]:
foo
# # Convert pickup_datetime to pandas DateTime
# df["pickup_datetime"] = pd.to_datetime(df["pickup_datetime"])
# # Convert UTC to EST
# df["pickup_datetime"] = df["pickup_datetime"].dt.tz_convert("US/Eastern")
# # Drop timezone suffix
# df["pickup_datetime"] = df["pickup_datetime"].dt.tz_localize(None)
# # foo["pickup_datetime"].dt.tz_localize(None)

,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude
0,2015-05-07 15:52:06,-73.999817,40.738354,-73.999512,40.723217
1,2009-07-17 16:04:56,-73.994355,40.728225,-73.994710,40.750325
2,2009-08-24 17:45:00,-74.005043,40.740770,-73.962565,40.772647
3,2009-06-26 04:22:21,-73.976124,40.790844,-73.965316,40.803349
4,2014-08-28 13:47:00,-73.925023,40.744085,-73.973082,40.761247
...,...,...,...,...,...
199995,2012-10-28 06:49:00,-73.987042,40.739367,-73.986525,40.740297
199996,2014-03-13 21:09:00,-73.984722,40.736837,-74.006672,40.739620
199997,2009-06-28 20:42:00,-73.986017,40.756487,-73.858957,40.692588
199998,2015-05-20 10:56:25,-73.997124,40.725452,-73.983215,40.695415


<IPython.core.display.Javascript object>

In [21]:
df.dtypes

pickup_datetime      datetime64[ns]
pickup_longitude            float64
pickup_latitude             float64
dropoff_longitude           float64
dropoff_latitude            float64
dtype: object

<IPython.core.display.Javascript object>

In [ ]:
def get_uber_data():
    uber_dataframe = load_and_clean_uber_data(UBER_DATA)
    add_distance_column(uber_dataframe)
    return uber_dataframe

### Processing Weather Data

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [ ]:
def clean_month_weather_data_hourly(csv_file):
    raise NotImplemented()

In [ ]:
def clean_month_weather_data_daily(csv_file):
    raise NotImplemented()

In [ ]:
def load_and_clean_weather_data():
    hourly_dataframes = []
    daily_dataframes = []
    
    # add some way to find all weather CSV files
    # or just add the name/paths manually
    weather_csv_files = ["TODO"]
    
    for csv_file in weather_csv_files:
        hourly_dataframe = clean_month_weather_data_hourly(csv_file)
        daily_dataframe = clean_month_weather_data_daily(csv_file)
        hourly_dataframes.append(hourly_dataframe)
        daily_dataframes.append(daily_dataframe)
        
    # create two dataframes with hourly & daily data from every month
    hourly_data = pd.concat(hourly_dataframes)
    daily_data = pd.concat(daily_dataframes)
    
    return hourly_data, daily_data

### Process All Data

_This is where you can actually execute all the required functions._

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [ ]:
taxi_data = get_and_clean_taxi_data()
uber_data = get_uber_data()
hourly_weather_data, daily_weather_data = load_and_clean_weather_data()

## Part 2: Storing Cleaned Data

_Write some prose that tells the reader what you're about to do here._

In [ ]:
engine = db.create_engine(DATABASE_URL)

In [ ]:
# if using SQL (as opposed to SQLAlchemy), define the commands 
# to create your 4 tables/dataframes
HOURLY_WEATHER_SCHEMA = """
TODO
"""

DAILY_WEATHER_SCHEMA = """
TODO
"""

TAXI_TRIPS_SCHEMA = """
TODO
"""

UBER_TRIPS_SCHEMA = """
TODO
"""

In [ ]:
# create that required schema.sql file
with open(DATABASE_SCHEMA_FILE, "w") as f:
    f.write(HOURLY_WEATHER_SCHEMA)
    f.write(DAILY_WEATHER_SCHEMA)
    f.write(TAXI_TRIPS_SCHEMA)
    f.write(UBER_TRIPS_SCHEMA)

In [ ]:
# create the tables with the schema files
with engine.connect() as connection:
    pass

### Add Data to Database

_**TODO:** Write some prose that tells the reader what you're about to do here._

In [ ]:
def write_dataframes_to_table(table_to_df_dict):
    raise NotImplemented()

In [ ]:
map_table_name_to_dataframe = {
    "taxi_trips": taxi_data,
    "uber_trips": uber_data,
    "hourly_weather": hourly_data,
    "daily_weather": daily_data,
}

In [ ]:
write_dataframes_to_table(map_table_name_to_dataframe)

## Part 3: Understanding the Data

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] For 01-2009 through 06-2015, what hour of the day was the most popular to take a yellow taxi? The result should have 24 bins.
* [ ] For the same time frame, what day of the week was the most popular to take an uber? The result should have 7 bins.
* [ ] What is the 95% percentile of distance traveled for all hired trips during July 2013?
* [ ] What were the top 10 days with the highest number of hired rides for 2009, and what was the average distance for each day?
* [ ] Which 10 days in 2014 were the windiest, and how many hired trips were made on those days?
* [ ] During Hurricane Sandy in NYC (Oct 29-30, 2012) and the week leading up to it, how many trips were taken each hour, and for each hour, how much precipitation did NYC receive and what was the sustained wind speed?

In [ ]:
def write_query_to_file(query, outfile):
    raise NotImplemented()

### Query N

_**TODO:** Write some prose that tells the reader what you're about to do here._

_Repeat for each query_

In [ ]:
QUERY_N = """
TODO
"""

In [ ]:
engine.execute(QUERY_N).fetchall()

In [ ]:
write_query_to_file(QUERY_N, "some_descriptive_name.sql")

## Part 4: Visualizing the Data

_A checklist of requirements to keep you on track. Remove this whole cell before submitting the project. The order of these tasks aren't necessarily the order in which they need to be done. It's okay to do them in an order that makes sense to you._

* [ ] Create an appropriate visualization for the first query/question in part 3
* [ ] Create a visualization that shows the average distance traveled per month (regardless of year - so group by each month). Include the 90% confidence interval around the mean in the visualization
* [ ] Define three lat/long coordinate boxes around the three major New York airports: LGA, JFK, and EWR (you can use bboxfinder to help). Create a visualization that compares what day of the week was most popular for drop offs for each airport.
* [ ] Create a heatmap of all hired trips over a map of the area. Consider using KeplerGL or another library that helps generate geospatial visualizations.
* [ ] Create a scatter plot that compares tip amount versus distance.
* [ ] Create another scatter plot that compares tip amount versus precipitation amount.

_Be sure these cells are executed so that the visualizations are rendered when the notebook is submitted._

### Visualization N

_**TODO:** Write some prose that tells the reader what you're about to do here._

_Repeat for each visualization._

_The example below makes use of the `matplotlib` library. There are other libraries, including `pandas` built-in plotting library, kepler for geospatial data representation, `seaborn`, and others._

In [ ]:
# use a more descriptive name for your function
def plot_visual_n(dataframe):
    figure, axes = plt.subplots(figsize=(20, 10))
    
    values = "..."  # use the dataframe to pull out values needed to plot
    
    # you may want to use matplotlib to plot your visualizations;
    # there are also many other plot types (other 
    # than axes.plot) you can use
    axes.plot(values, "...")
    # there are other methods to use to label your axes, to style 
    # and set up axes labels, etc
    axes.set_title("Some Descriptive Title")
    
    plt.show()

In [ ]:
def get_data_for_visual_n():
    # Query SQL database for the data needed.
    # You can put the data queried into a pandas dataframe, if you wish
    raise NotImplemented()

In [ ]:
some_dataframe = get_data_for_visual_n()
plot_visual_n(some_dataframe)